# 02 - Event Study Analysis

This notebook demonstrates **Event Study Analysis** - a standard finance technique to measure if a specific event caused an abnormal market reaction.

---

## What is an Event Study?

Event studies are the **gold standard in finance** for answering: *"Did this event affect the stock/market?"*

**Real-world applications:**
- Did the Fed rate announcement move stocks?
- Did the CEO departure hurt the company?
- Did the sanctions affect oil prices?
- Did the earnings beat expectations?

**The key insight:** Markets are noisy - prices move every day. To isolate an event's effect, we need to compare *actual* returns to *expected* returns.

---

## The Methodology

```
         ESTIMATION WINDOW           GAP    EVENT WINDOW
    |---------------------------|   |---|  |----------|
    t-40                       t-6  t-2 t-1  t=0  t+1 ... t+5
    
    "What's normal?"                      "What happened?"
```

1. **Estimation Window** (t-40 to t-6): Calculate "normal" daily returns
2. **Gap** (t-5 to t-2): Buffer to avoid event contamination
3. **Event Window** (t-1 to t+5): Measure actual returns around the event
4. **Abnormal Return** = Actual - Expected (for each day)
5. **CAR** = Cumulative Abnormal Return (sum of all abnormal returns)
6. **Statistical Test**: Is CAR significantly different from zero?

---

## Key Concepts You'll Learn

| Concept | Description | Why It Matters |
|---------|-------------|----------------|
| **Abnormal Return (AR)** | Return beyond what's expected | Isolates event impact from normal noise |
| **CAR** | Sum of ARs over event window | Total cumulative impact |
| **t-statistic** | CAR / Standard Error | Measures "how many SEs from zero" |
| **p-value** | Probability of seeing CAR by chance | <0.05 means statistically significant |
| **Confidence Interval** | Range of plausible CAR values | 95% CI tells you uncertainty |

---

We'll compare our **learning version** (manual implementation) with the **production version** (using scipy).

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# IMPORTS
# ═══════════════════════════════════════════════════════════════════════════════
#
# Standard practice: Group imports by category
#   1. Standard library (sys, pathlib, datetime)
#   2. Third-party (pandas, numpy, matplotlib)
#   3. Project-specific (src.analysis)
#
# ═══════════════════════════════════════════════════════════════════════════════

import sys
from pathlib import Path
from datetime import date, timedelta

# Add project root to Python path
# This allows us to import from src/ even when running from notebooks/
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Matplotlib style
# 'seaborn-v0_8-whitegrid' provides clean, professional plots
# The grid helps with reading values off charts
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

print("Imports successful!")

## 1. Understanding Event Studies

### Why Not Just Look at Returns?

You might think: *"If oil dropped 3% on the day of a geopolitical event, the event caused the drop, right?"*

**Not necessarily.** Here's why:

1. **Oil is volatile** - it might drop 3% on a random Tuesday
2. **Multiple factors** - interest rates, inventory reports, demand forecasts all affect oil
3. **Market anticipation** - traders might have priced in the event beforehand

**The solution:** Compare the *actual* return to what we'd *expect* based on recent history.

### The Timeline Visualization

Let's visualize the event study timeline:

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# EVENT STUDY TIMELINE VISUALIZATION
# ═══════════════════════════════════════════════════════════════════════════════
#
# This visualization shows the three critical periods:
#
# 1. ESTIMATION WINDOW (blue):
#    - Used to calculate "normal" market behavior
#    - Typically 30-250 trading days
#    - We use the MEAN return as our "expected" return
#    - We use the STANDARD DEVIATION for statistical tests
#
# 2. GAP PERIOD:
#    - Buffer between estimation and event windows
#    - Prevents event anticipation from contaminating "normal" estimates
#    - Usually 2-5 days before the event
#
# 3. EVENT WINDOW (red):
#    - The period we're actually studying
#    - Includes days before (anticipation) and after (reaction)
#    - Typically t-1 to t+5 or t-2 to t+10
#
# ═══════════════════════════════════════════════════════════════════════════════

fig, ax = plt.subplots(figsize=(14, 4))

# Timeline base
days = list(range(-40, 11))
ax.axhline(y=0, color='black', linewidth=2)

# Estimation window (where we learn "normal" behavior)
ax.axvspan(-40, -6, alpha=0.3, color='blue', label='Estimation Window')
ax.text(-23, 0.3, 'ESTIMATION WINDOW\n(Calculate normal returns)\n\nμ = mean return\nσ = std deviation', 
        ha='center', fontsize=10)

# Event window (where we measure impact)
ax.axvspan(-1, 5, alpha=0.3, color='red', label='Event Window')
ax.text(2, 0.3, 'EVENT\nWINDOW\n\nMeasure\nactual returns', ha='center', fontsize=10)

# Event day marker
ax.axvline(x=0, color='green', linewidth=3, label='Event Day (t=0)')
ax.annotate('EVENT!', xy=(0, 0), xytext=(0, -0.5), fontsize=12, ha='center',
            arrowprops=dict(arrowstyle='->', color='green'))

# Gap annotation
ax.annotate('Gap', xy=(-3.5, 0.1), fontsize=9, ha='center', color='gray')

ax.set_xlim(-45, 15)
ax.set_ylim(-0.8, 0.8)
ax.set_xlabel('Days Relative to Event (t=0 is event day)')
ax.set_yticks([])
ax.set_title('Event Study Timeline: Separating "Normal" from "Event Impact"')
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

print("\nKey insight: The ESTIMATION WINDOW must be BEFORE the event,")
print("so we're measuring 'normal' behavior unaffected by the event!")

## 2. Learning Version: Step-by-Step

Our **learning version** implements everything from scratch so you can see exactly how it works.

### The Algorithm

```python
# Pseudocode for event study

# Step 1: Estimate "normal" returns
expected_return = mean(estimation_window_returns)
std_dev = std(estimation_window_returns)

# Step 2: Calculate abnormal returns for each day in event window
for day in event_window:
    AR[day] = actual_return[day] - expected_return

# Step 3: Sum up the abnormal returns
CAR = sum(AR)  # Cumulative Abnormal Return

# Step 4: Test if CAR is statistically significant
SE = std_dev * sqrt(len(event_window))  # Standard Error of CAR
t_stat = CAR / SE
p_value = 2 * (1 - t_distribution.cdf(abs(t_stat)))

# Significant if p < 0.05
```

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# LEARNING VERSION: EventStudy CLASS
# ═══════════════════════════════════════════════════════════════════════════════
#
# This is our EDUCATIONAL implementation that shows all the math.
#
# Key parameters:
#   - estimation_window: Days to calculate "normal" (30 is common)
#   - event_window_before: Days before event to include (captures anticipation)
#   - event_window_after: Days after event to include (captures full reaction)
#   - significance_level: Alpha for hypothesis test (0.05 = 95% confidence)
#
# Academic standards:
#   - estimation_window: 120-250 days for stocks, 30-60 for commodities
#   - event_window: Depends on event type (earnings: 3 days, M&A: 20+ days)
#
# ═══════════════════════════════════════════════════════════════════════════════

from src.analysis.event_study import EventStudy, EventStudyResult, explain_result

# Create analyzer with explicit parameters
study = EventStudy(
    estimation_window=30,     # 30 trading days to calculate "normal" returns
    event_window_before=1,    # Include 1 day before (might show anticipation)
    event_window_after=5,     # Include 5 days after (capture full reaction)
    significance_level=0.05,  # Standard 95% confidence level
)

print("Event Study Configuration")
print("=" * 50)
print(f"  Estimation window: {study.estimation_window} trading days")
print(f"  Event window: [{-study.event_window_before}, +{study.event_window_after}] days")
print(f"  Total event window days: {study.event_window_before + study.event_window_after + 1}")
print(f"  Significance level (α): {study.significance_level}")
print(f"  Confidence level: {(1 - study.significance_level) * 100:.0f}%")
print()
print("Interpretation:")
print(f"  - We'll use {study.estimation_window} days of data to establish 'normal'")
print(f"  - We'll measure returns from day {-study.event_window_before} to day +{study.event_window_after}")
print(f"  - If p-value < {study.significance_level}, we conclude the event had a real effect")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# RUNNING AN EVENT STUDY
# ═══════════════════════════════════════════════════════════════════════════════
#
# We pick a date from about 2 weeks ago to ensure:
#   1. We have the required estimation window data (30 days before)
#   2. We have the full event window data (5 days after)
#
# The study.analyze_event() method:
#   1. Fetches market data from the database
#   2. Calculates expected returns from estimation window
#   3. Computes abnormal returns for each day in event window
#   4. Sums them into CAR (Cumulative Abnormal Return)
#   5. Performs t-test for statistical significance
#
# ═══════════════════════════════════════════════════════════════════════════════

# Choose a date with enough data on both sides
event_date = date.today() - timedelta(days=14)
symbol = "CL=F"  # Crude Oil Futures

print(f"Event Study: {symbol} on {event_date}")
print("=" * 50)
print()
print("Data requirements:")
print(f"  - Need data from: {event_date - timedelta(days=45)} (estimation start)")
print(f"  - Through: {event_date + timedelta(days=7)} (event window end)")
print()

# Run the event study
result = study.analyze_event(
    event_id=1,           # Just an identifier
    symbol=symbol,        # Which market to analyze
    event_date=event_date # The date of the "event"
)

if result:
    # The explain_result function provides a human-readable summary
    print(explain_result(result))
else:
    print("No result returned.")
    print()
    print("Troubleshooting:")
    print("  1. Make sure you've run the data ingestion scripts")
    print("  2. Verify the database has data for this symbol and date range")
    print("  3. Try: python scripts/ingest_market_data.py")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# VISUALIZING ABNORMAL RETURNS
# ═══════════════════════════════════════════════════════════════════════════════
#
# Two views of the same data:
#
# LEFT PLOT - Daily Abnormal Returns:
#   - Each bar shows AR for that day
#   - Green = positive abnormal return (better than expected)
#   - Red = negative abnormal return (worse than expected)
#   - Day 0 (blue dashed line) is the event day
#
# RIGHT PLOT - Cumulative Abnormal Return (CAR):
#   - Shows the running total of ARs
#   - The FINAL value is what we test for significance
#   - Shape tells you about timing:
#     - Jump ON day 0 → immediate reaction
#     - Gradual slope → delayed/prolonged reaction
#     - Jump BEFORE day 0 → information leaked early
#
# ═══════════════════════════════════════════════════════════════════════════════

if result:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Create day labels relative to event
    days = list(range(-study.event_window_before, study.event_window_after + 1))
    abnormal_returns = result.abnormal_returns[:len(days)]
    
    # ─── LEFT PLOT: Daily Abnormal Returns ───
    colors = ['green' if r > 0 else 'red' for r in abnormal_returns]
    
    axes[0].bar(
        days[:len(abnormal_returns)], 
        [r * 100 for r in abnormal_returns],  # Convert to percentage
        color=colors, 
        alpha=0.7, 
        edgecolor='black'
    )
    axes[0].axhline(y=0, color='black', linestyle='-')
    axes[0].axvline(x=0, color='blue', linestyle='--', linewidth=2, label='Event Day')
    axes[0].set_xlabel('Days Relative to Event')
    axes[0].set_ylabel('Abnormal Return (%)')
    axes[0].set_title(f'Daily Abnormal Returns: {symbol}\n(Actual - Expected)')
    axes[0].legend()
    
    # Add value labels on bars
    for i, (day, ar) in enumerate(zip(days[:len(abnormal_returns)], abnormal_returns)):
        if abs(ar) > 0.005:  # Only label if > 0.5%
            axes[0].annotate(f'{ar*100:.1f}%', 
                           xy=(day, ar*100), 
                           ha='center', 
                           va='bottom' if ar > 0 else 'top',
                           fontsize=8)
    
    # ─── RIGHT PLOT: Cumulative Abnormal Return ───
    car_cumulative = np.cumsum(abnormal_returns) * 100
    
    axes[1].plot(
        days[:len(car_cumulative)], 
        car_cumulative, 
        marker='o', 
        linewidth=2, 
        markersize=8,
        color='steelblue'
    )
    axes[1].axhline(y=0, color='black', linestyle='-')
    axes[1].axvline(x=0, color='blue', linestyle='--', linewidth=2, label='Event Day')
    
    # Fill area to show magnitude
    axes[1].fill_between(
        days[:len(car_cumulative)], 
        0, 
        car_cumulative, 
        alpha=0.3,
        color='green' if result.car > 0 else 'red'
    )
    
    axes[1].set_xlabel('Days Relative to Event')
    axes[1].set_ylabel('Cumulative Abnormal Return (%)')
    axes[1].set_title(f'CAR Over Event Window\nFinal CAR: {result.car*100:.2f}% (p={result.p_value:.3f})')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Interpretation
    print("\nHow to read these charts:")
    print("─" * 50)
    print("LEFT: Each bar shows how much the return DEVIATED from expected")
    print("RIGHT: The cumulative sum - this is what we test for significance")
    if result.is_significant:
        print(f"\n→ The {result.car*100:.2f}% CAR is STATISTICALLY SIGNIFICANT")
        print(f"  There's only a {result.p_value*100:.1f}% chance this happened randomly")
    else:
        print(f"\n→ The {result.car*100:.2f}% CAR is NOT statistically significant")
        print(f"  This could easily be random noise (p={result.p_value:.3f})")

## 3. Production Version: Using scipy

The **production version** uses scipy for more robust statistical tests:

| Feature | Learning Version | Production Version |
|---------|-----------------|--------------------|
| Statistical test | Basic t-test | scipy.stats.ttest_1samp |
| Confidence interval | Not included | Computed automatically |
| Non-parametric test | Not included | Wilcoxon signed-rank |
| Code complexity | ~100 lines | ~50 lines |

### Why Use Both?

- **Learning version**: Understand the math, great for interviews
- **Production version**: Reliable, tested, what you'd use at work

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# PRODUCTION VERSION: Quick Analysis
# ═══════════════════════════════════════════════════════════════════════════════
#
# The run_quick_event_study() function is a convenience wrapper that:
#   1. Creates a ProductionEventStudy with default parameters
#   2. Runs the analysis
#   3. Returns a formatted summary string
#
# This is the "just give me the answer" approach - perfect for quick checks.
#
# ═══════════════════════════════════════════════════════════════════════════════

from src.analysis.production_event_study import ProductionEventStudy, run_quick_event_study

# Quick one-liner for fast analysis
summary = run_quick_event_study(symbol, event_date)

if summary:
    print("Production Version - Quick Summary")
    print("=" * 50)
    print(summary)
else:
    print("No result - insufficient data")
    print("Ensure the database is populated with market data.")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# PRODUCTION VERSION: Full Details
# ═══════════════════════════════════════════════════════════════════════════════
#
# For more control, instantiate ProductionEventStudy directly.
#
# Additional features over learning version:
#
# 1. CONFIDENCE INTERVAL (ci_lower, ci_upper):
#    - 95% CI tells you the range of plausible CAR values
#    - If CI doesn't include 0, the effect is significant
#
# 2. WILCOXON SIGNED-RANK TEST (wilcoxon_p):
#    - Non-parametric alternative to t-test
#    - Doesn't assume returns are normally distributed
#    - More robust for small samples or fat-tailed returns
#
# Interview tip: Mentioning "I'd also run a Wilcoxon test because
# returns have fat tails" shows statistical sophistication!
#
# ═══════════════════════════════════════════════════════════════════════════════

prod_study = ProductionEventStudy()
prod_result = prod_study.analyze_event(event_id=1, symbol=symbol, event_date=event_date)

if prod_result:
    print("Production Version - Full Results")
    print("=" * 50)
    print()
    print("Core Metrics:")
    print(f"  CAR: {prod_result.car*100:.4f}%")
    print(f"  t-statistic: {prod_result.t_statistic:.4f}")
    print(f"  p-value: {prod_result.p_value:.4f}")
    print()
    print("Confidence Interval:")
    print(f"  95% CI: [{prod_result.ci_lower*100:.4f}%, {prod_result.ci_upper*100:.4f}%]")
    ci_includes_zero = prod_result.ci_lower <= 0 <= prod_result.ci_upper
    print(f"  CI includes zero: {ci_includes_zero} {'→ NOT significant' if ci_includes_zero else '→ SIGNIFICANT'}")
    print()
    print("Statistical Significance:")
    print(f"  Is significant (α=0.05): {prod_result.is_significant}")
    
    if prod_result.wilcoxon_p:
        print()
        print("Non-parametric Test:")
        print(f"  Wilcoxon signed-rank p-value: {prod_result.wilcoxon_p:.4f}")
        print("  (Doesn't assume normal distribution - more robust for financial returns)")

## 4. Analyzing Multiple Events

Real-world event studies often analyze **multiple events** simultaneously:

- Compare how different markets responded to the same event
- Compare how the same market responded to different events
- Build up statistical power by aggregating many events

### Cross-Market Comparison

Let's see how different markets responded to our event date:

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# MULTI-MARKET COMPARISON
# ═══════════════════════════════════════════════════════════════════════════════
#
# We analyze the same "event" across multiple markets:
#
# - CL=F: Crude Oil Futures (sensitive to geopolitical events)
# - GC=F: Gold Futures (safe haven asset)
# - SPY: S&P 500 ETF (broad US equity market)
# - ^VIX: Volatility Index (fear gauge)
#
# Expectations for a NEGATIVE geopolitical event:
# - Oil: ↑ (supply concerns) or ↓ (demand fears)
# - Gold: ↑ (flight to safety)
# - SPY: ↓ (risk-off sentiment)
# - VIX: ↑ (increased fear/uncertainty)
#
# Note: VIX moves INVERSELY to market sentiment!
#
# ═══════════════════════════════════════════════════════════════════════════════

symbols_to_analyze = ['CL=F', 'GC=F', 'SPY', '^VIX']
results_list = []

print("Analyzing multiple markets...")
print("─" * 50)

for sym in symbols_to_analyze:
    result = prod_study.analyze_event(event_id=0, symbol=sym, event_date=event_date)
    if result:
        results_list.append({
            'symbol': sym,
            'date': event_date,
            'car_pct': result.car * 100,
            't_stat': result.t_statistic,
            'p_value': result.p_value,
            'significant': result.is_significant,
        })
        status = "✓ SIGNIFICANT" if result.is_significant else "  not significant"
        print(f"  {sym}: CAR = {result.car*100:+.2f}% (p={result.p_value:.3f}) {status}")
    else:
        print(f"  {sym}: No data available")

# Display as DataFrame
if results_list:
    print()
    results_df = pd.DataFrame(results_list)
    print("\nEvent Study Results Across Markets")
    print("=" * 60)
    display(results_df.style.format({
        'car_pct': '{:+.2f}%',
        't_stat': '{:.3f}',
        'p_value': '{:.4f}',
    }))
else:
    print("\nNo results - ensure data is ingested for these symbols")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# VISUALIZATION: CAR COMPARISON ACROSS MARKETS
# ═══════════════════════════════════════════════════════════════════════════════
#
# Visual encoding:
#   - Bar HEIGHT = CAR magnitude
#   - Bar COLOR = Direction (green = positive, red = negative)
#   - Bar OPACITY = Statistical significance (solid = significant, faded = not)
#
# This visualization immediately tells you:
#   1. Which markets were most affected
#   2. Direction of the effect
#   3. Which effects are statistically meaningful
#
# ═══════════════════════════════════════════════════════════════════════════════

if results_list:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Determine colors based on CAR direction
    colors = ['green' if r['car_pct'] > 0 else 'red' for r in results_list]
    # Determine opacity based on significance
    alphas = [1.0 if r['significant'] else 0.4 for r in results_list]
    
    # Create bars
    bars = ax.bar(
        [r['symbol'] for r in results_list],
        [r['car_pct'] for r in results_list],
        color=colors,
        edgecolor='black',
    )
    
    # Apply alpha (opacity) for significance
    for bar, alpha in zip(bars, alphas):
        bar.set_alpha(alpha)
    
    # Reference line at zero
    ax.axhline(y=0, color='black', linestyle='-', linewidth=1)
    
    # Add value labels
    for bar, r in zip(bars, results_list):
        height = bar.get_height()
        va = 'bottom' if height > 0 else 'top'
        offset = 0.1 if height > 0 else -0.1
        sig_marker = '*' if r['significant'] else ''
        ax.annotate(f"{height:+.1f}%{sig_marker}",
                   xy=(bar.get_x() + bar.get_width()/2, height + offset),
                   ha='center', va=va, fontsize=11, fontweight='bold')
    
    ax.set_xlabel('Market', fontsize=12)
    ax.set_ylabel('Cumulative Abnormal Return (%)', fontsize=12)
    ax.set_title(f'Event Impact Across Markets ({event_date})\n(Faded = Not Statistically Significant, * = Significant)',
                fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    print("\nInterpretation Guide:")
    print("─" * 50)
    print("• Solid bars: Statistically significant (p < 0.05)")
    print("• Faded bars: Not significant (could be random noise)")
    print("• Only draw conclusions from SOLID bars!")

## 5. The Math Behind Event Studies

Let's walk through the calculations **step by step** so you understand exactly what's happening.

### The Four Steps

1. **Estimate normal returns** from the estimation window
2. **Calculate abnormal returns** for each event day
3. **Sum into CAR** (Cumulative Abnormal Return)
4. **Test significance** using a t-test

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# STEP 0: GET RAW DATA
# ═══════════════════════════════════════════════════════════════════════════════
#
# First, we need to fetch the raw return data and split it into:
#   - Estimation window: For calculating "normal" returns
#   - Event window: For measuring the event's impact
#
# We use log returns because they're:
#   - Additive across time (daily log returns sum to period return)
#   - More normally distributed than simple returns
#   - Standard in academic finance
#
# ═══════════════════════════════════════════════════════════════════════════════

from src.db.queries import get_market_data
from src.db.connection import get_session

# Define our windows
estimation_start = event_date - timedelta(days=45)  # Start of estimation
estimation_end = event_date - timedelta(days=2)     # End of estimation (gap before event)
event_start = event_date - timedelta(days=1)        # Event window start
event_end = event_date + timedelta(days=5)          # Event window end

print("Data Windows")
print("=" * 50)
print(f"Estimation window: {estimation_start} to {estimation_end}")
print(f"Event window:      {event_start} to {event_end}")
print()

with get_session() as session:
    data = get_market_data(session, 'CL=F', estimation_start, event_end)
    
    if data:
        # Convert to DataFrame
        df = pd.DataFrame([
            {'date': d.date, 'return': d.log_return}
            for d in data
        ]).dropna()
        
        print(f"Total data points loaded: {len(df)}")
        
        # Split into windows
        estimation_df = df[(df['date'] >= estimation_start) & (df['date'] <= estimation_end)]
        event_df = df[(df['date'] >= event_start) & (df['date'] <= event_end)]
        
        print(f"Estimation window: {len(estimation_df)} trading days")
        print(f"Event window: {len(event_df)} trading days")
    else:
        print("No data available. Please run the ingestion scripts.")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# STEP 1: ESTIMATE NORMAL RETURNS
# ═══════════════════════════════════════════════════════════════════════════════
#
# From the estimation window, we calculate:
#   - μ (mean): The expected daily return
#   - σ (std): The typical daily volatility
#
# These become our baseline for "normal" market behavior.
#
# Why the mean? 
#   - Simple and unbiased
#   - Alternative: CAPM-adjusted returns (but needs market data)
#   - For short windows, mean works well
#
# ═══════════════════════════════════════════════════════════════════════════════

if 'estimation_df' in dir() and len(estimation_df) > 0:
    expected_return = estimation_df['return'].mean()
    std_dev = estimation_df['return'].std()
    
    print("STEP 1: Estimate Normal Returns")
    print("=" * 50)
    print()
    print("From the estimation window, we calculate:")
    print()
    print(f"  Expected (mean) daily return: {expected_return*100:.4f}%")
    print(f"  Standard deviation:           {std_dev*100:.4f}%")
    print()
    print("Interpretation:")
    print(f"  - On a 'normal' day, oil returns about {expected_return*100:.4f}%")
    print(f"  - With typical daily swings of ±{std_dev*100:.2f}%")
    print(f"  - A 2σ move (unusual) would be ±{2*std_dev*100:.2f}%")
else:
    print("No estimation data available.")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# STEP 2: CALCULATE ABNORMAL RETURNS
# ═══════════════════════════════════════════════════════════════════════════════
#
# Abnormal Return (AR) = Actual Return - Expected Return
#
# This isolates the "abnormal" component of each day's return.
#
# Interpretation:
#   AR > 0: Market did BETTER than expected (positive surprise)
#   AR < 0: Market did WORSE than expected (negative surprise)
#   AR ≈ 0: Market did about as expected (no abnormal behavior)
#
# ═══════════════════════════════════════════════════════════════════════════════

if 'event_df' in dir() and len(event_df) > 0:
    # Calculate abnormal returns
    abnormal = event_df['return'] - expected_return
    
    print("STEP 2: Calculate Abnormal Returns")
    print("=" * 50)
    print()
    print("Formula: AR = Actual Return - Expected Return")
    print()
    print(f"{'Day':<6} {'Date':<12} {'Actual':>10} {'Expected':>10} {'Abnormal':>10}")
    print("─" * 50)
    
    for i, (_, row) in enumerate(event_df.iterrows()):
        ar = row['return'] - expected_return
        day_label = f"t{i-1:+d}" if i != 1 else "t=0"  # Adjust based on window
        print(f"{day_label:<6} {str(row['date']):<12} {row['return']*100:>9.4f}% {expected_return*100:>9.4f}% {ar*100:>9.4f}%")
    
    print("─" * 50)
    print()
    print("Interpretation:")
    max_ar = abnormal.max()
    min_ar = abnormal.min()
    print(f"  - Largest positive abnormal return: {max_ar*100:+.4f}%")
    print(f"  - Largest negative abnormal return: {min_ar*100:+.4f}%")
else:
    print("No event window data available.")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# STEP 3: CALCULATE CUMULATIVE ABNORMAL RETURN (CAR)
# ═══════════════════════════════════════════════════════════════════════════════
#
# CAR = Sum of all abnormal returns in the event window
#
# This is the TOTAL abnormal impact of the event.
#
# Why sum instead of average?
#   - Sum captures total impact over the window
#   - If event has multi-day effect, we want to capture all of it
#   - Average would dilute a one-day spike
#
# ═══════════════════════════════════════════════════════════════════════════════

if 'abnormal' in dir():
    car = abnormal.sum()
    
    print("STEP 3: Calculate Cumulative Abnormal Return (CAR)")
    print("=" * 50)
    print()
    print("Formula: CAR = Σ AR (sum of all abnormal returns)")
    print()
    print("Calculation:")
    ar_values = [f"{ar*100:+.4f}%" for ar in abnormal]
    print(f"  CAR = {' + '.join(ar_values)}")
    print(f"  CAR = {car*100:+.4f}%")
    print()
    print("Interpretation:")
    if car > 0:
        print(f"  - The market returned {car*100:.2f}% MORE than expected")
        print(f"  - This is a POSITIVE abnormal return")
    else:
        print(f"  - The market returned {abs(car)*100:.2f}% LESS than expected")
        print(f"  - This is a NEGATIVE abnormal return")
else:
    print("No abnormal returns calculated.")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# STEP 4: TEST STATISTICAL SIGNIFICANCE
# ═══════════════════════════════════════════════════════════════════════════════
#
# The key question: Is the CAR significantly different from zero?
#
# We use a t-test:
#   1. Calculate Standard Error of CAR: SE = σ × √n
#      - σ = std dev from estimation window
#      - n = number of days in event window
#   2. Calculate t-statistic: t = CAR / SE
#      - Measures "how many standard errors is CAR from zero"
#   3. Calculate p-value from t-distribution
#      - Probability of seeing this t-stat by chance
#
# Decision rule: If p < 0.05, reject null hypothesis (CAR = 0)
#
# ═══════════════════════════════════════════════════════════════════════════════

if 'car' in dir() and 'std_dev' in dir():
    from scipy import stats
    
    n_days = len(event_df)
    
    # Standard Error of CAR
    # Why sqrt(n)? Because variance of sum = n × variance of individual
    # So SE of sum = sqrt(n) × SE of individual = sqrt(n) × σ
    se_car = std_dev * np.sqrt(n_days)
    
    # t-statistic
    t_stat = car / se_car
    
    # p-value (two-tailed test)
    # We use t-distribution with (estimation_window - 1) degrees of freedom
    df = len(estimation_df) - 1
    p_value = 2 * (1 - stats.t.cdf(abs(t_stat), df=df))
    
    print("STEP 4: Test Statistical Significance")
    print("=" * 50)
    print()
    print("Hypothesis Test:")
    print("  H₀ (null): CAR = 0 (no abnormal return)")
    print("  H₁ (alt):  CAR ≠ 0 (significant abnormal return)")
    print()
    print("Calculations:")
    print(f"  Standard Error = σ × √n = {std_dev*100:.4f}% × √{n_days}")
    print(f"                 = {se_car*100:.4f}%")
    print()
    print(f"  t-statistic = CAR / SE = {car*100:.4f}% / {se_car*100:.4f}%")
    print(f"              = {t_stat:.4f}")
    print()
    print(f"  Degrees of freedom: {df}")
    print(f"  p-value (two-tailed): {p_value:.4f}")
    print()
    print("─" * 50)
    print("RESULT:")
    print("─" * 50)
    if p_value < 0.05:
        print(f"  ✓ STATISTICALLY SIGNIFICANT at 95% confidence!")
        print(f"  - p-value ({p_value:.4f}) < 0.05")
        print(f"  - We REJECT H₀: The CAR is significantly different from zero")
        print(f"  - The event likely had a real effect on the market")
    else:
        print(f"  ✗ NOT statistically significant")
        print(f"  - p-value ({p_value:.4f}) ≥ 0.05")
        print(f"  - We FAIL TO REJECT H₀: Cannot conclude CAR differs from zero")
        print(f"  - This could easily be random market noise")

## Summary

**Event Study Analysis** answers: *"Did this event move the market more than expected?"*

---

### Key Concepts

| Metric | Formula | Interpretation |
|--------|---------|----------------|
| **Expected Return** | μ = mean(estimation_window) | "Normal" daily return |
| **Abnormal Return** | AR = Actual - Expected | Deviation from normal |
| **CAR** | Σ AR | Total abnormal impact |
| **Standard Error** | SE = σ × √n | Uncertainty in CAR |
| **t-statistic** | t = CAR / SE | # of SEs from zero |
| **p-value** | P(|t| > observed) | Probability by chance |

---

### Decision Framework

```
p < 0.05  →  Significant     →  Event likely had real effect
p ≥ 0.05  →  Not significant →  Could be random noise
```

---

### Two Implementations

| Version | Class | Use Case |
|---------|-------|----------|
| **Learning** | `EventStudy` | Understand the math, interviews |
| **Production** | `ProductionEventStudy` | Real work, adds CI & Wilcoxon |

---

### Common Pitfalls

1. **Confounding events**: Another event happened in your window
2. **Data snooping**: Picking dates after seeing the data
3. **Short windows**: May miss delayed reactions
4. **Long windows**: May capture unrelated noise

---

**Next:** See `03_anomaly_detection.ipynb` to find unusual market-event patterns.